# Dijkstra算法
一个有向图，问：从出发点到各节点的最短距离，返回表。(可以出现距离正无穷)

分析：
- 先准备一张初始表，A—A的距离为0，A—X的距离为Inf
- 选取初始表中最小的那个值(很明显应该选A-A),以这个A为初始点开始找寻到各节点的最短路径(只跳转一次)，如果这个路径的距离加上自己本身的值是小于表中的值，则更新表。得到表二
- 再从表二中选取最小的值(注意不能选取上一步的点)，以此节点开始跳转一次，计算路径的距离并加上自己本身的值，如果这个值小于表中这个节点的值则更新。
- 重复以上步骤直到最后一个节点被使用(usedSet中包含了所有节点)。
- 按步骤运行可以之后知道，其实usedSet储存了已经被遍历过路径的点了，起点startNote到usedSet中的点最短路径已经被updateMap记录了，不可能会再有其他路径能到useSet中的点距离能小于updateMap记录的距离了，因此可以直接跳过对usedSet中点的更新（不跳过也可以）

In [ ]:
# 通过字典来得到权有向图
graph = {
    "A":{"B":5, "C":1},              # 字典，和A相连的是B(5)，C(1)
    "B":{"A":5, "C":2, "D":1},       # 字典，和B相连的是A(5)，C(2),D(1)
    "C":{"A":1, "B":2, "D":4, "E":8},# 字典，和C相连的是A(2)，B(2),D(4),E(8)
    "D":{"B":1, "C":4, "E":3, "F":6},# 字典，和D相连的是B(1)，C(4),E(3),F(8)
    "E":{"C":8, "D":3},              # 字典，和E相连的是C(8)，D(3)
    "F":{"D":6}                      # 字典，和F相连的是D(6)
}

# 将返回当前updateMap中最小的点以及其基准距离
def getMinNote(updateMap, usedSet):
    '''
    usedSet: set()用于记录使用过的点
    updateMap: 到各节点的距离字典
    '''
    minimun = float("inf") 
    minNote = None 
    for i in updateMap :
       if (i not in usedSet) and (updateMap[i] < minimun):
           minNote = i 
           minimun = updateMap[i]
    return minNote, minimun

# Dijkstra算法
def Dijkstra(startNote, graph):
    updateMap = {startNote:0}    #放入初始点
    usedSet = set()  #记录使用过的点
    minNote, distance = getMinNote(updateMap, usedSet)

    while minNote != None:      # 当useSet中已经完成了所有节点，则不用再循环了
        edges = graph[minNote]  # 得到的是一个字典，表示minNote能到各个点的距离{X:int...}
        for note in edges:
            if note not in updateMap : # 如果更新字典中不存在这条边
                updateMap[note] = edges[note] + distance
            else :
                # if条件可以删掉
                if note not in usedSet: # startNote到useSet中的点的最短路径已经被updateMap储存了，不可能有更短的路径了，因此可以不用更新了
                    updateMap[note] = min(distance+edges[note], updateMap[note])

        usedSet.add(minNote)
        minNote, distance = getMinNote(updateMap, usedSet)

    return updateMap
    
Dijkstra("A", graph)


## Dijkstra算法改进
采用小根堆来存储记录，这样就不用遍历整个需要更新的字典来找到最小的距离了。

小根堆的需求：需要自己能update，ignore，主要到这里需要在小根堆的这个类中创造一个heapMap来记录节点的位置，因为这个堆的每个节点会更改，因此每次更改之后要重新排序。

在python中自带的heap可以创建一个元组堆（value,key），也就是把带数字的元组类型推入堆中，并且我们可以通过识别key来更改value的值，并且仍然是堆。实际上就做到了上述的需求。常见一个堆pqueue用来存放没有被完全更新结束的节点，更新结束的意思是当一个节点从pqueue中取出的时候，其实就意味着这个节点已经被更新完毕了。

In [ ]:
from heapq import * 

def Dijkstra(startNote, graph):
    updateMap ={startNote:0}    # 放入初始点
    usedSet = set()  # 记录使用过的点
    pqueue = []     # 用来存放还没被完全更新的节点
    heappush(pqueue, (0, startNote)) 

    while pqueue != []:      # 当pqueue为空时意味着已经被更新完毕了，则不用再循环了
        distance, minNote = heappop(pqueue)
        edges = graph[minNote]
        for note in edges:
            if note not in updateMap : # 如果updateMap中不存在这条边
                updateMap[note] = edges[note] + distance
                heappush(pqueue, (updateMap[note], note))  # 需要把待更新的点压进pqueue
            else :
                if note not in usedSet: # useSet里面的节点不能被更新
                    updateMap[note] = min(distance+edges[note], updateMap[note])
                    heappush(pqueue, (updateMap[note], note))  # 需要把待更新的点压进pqueue

        usedSet.add(minNote)
    return updateMap
    
graph = {
    "A":{"B":5, "C":1},              # 字典，和A相连的是B(5)，C(1)
    "B":{"A":5, "C":2, "D":1},       # 字典，和B相连的是A(5)，C(2),D(1)
    "C":{"A":1, "B":2, "D":4, "E":8},# 字典，和C相连的是A(2)，B(2),D(4),E(8)
    "D":{"B":1, "C":4, "E":3, "F":6},# 字典，和D相连的是B(1)，C(4),E(3),F(8)
    "E":{"C":8, "D":3},              # 字典，和E相连的是C(8)，D(3)
    "F":{"D":6}                      # 字典，和F相连的是D(6)
}

Dijkstra("A", graph)

# 字符串匹配算法（KMP和BF）
- 解决的问题：给定两个串S=“s1s2s3 …sn”和T=“t1t2t3 …tn”，在主串S中寻找子串T的过程叫做模式匹配，T称为模式
- 朴素模式匹配BF：现在假设现在主串S匹配到i位置，模式串T匹配到j位置
    - 如果当前字符匹配成功（即S\[i\] = T\[j\]），则i++，j++，继续匹配下一个字符；
    - 如果失配（即S\[i\] != T\[j\]），令i=i-j+1，j = 0。相当于每次匹配失败时，i回溯到本次失配起始字符的下一个字符，j回溯到0。
- KMP算法的改进：每当从某个起始位置开始一趟比较后，在匹配过程中出现失配，不回溯i，而是利用已经得到的部分匹配结果，将一种假想的位置定位“指针”在模式上向右滑动尽可能远的一段距离到某个位置后，继续按规则进行下一次的比较。
- KMP算法流程：规定i是主串S的下标，j是模式T的下标。现在假设现在主串S匹配到i位置，模式串T匹配到j位置。
    - 如果j = -1，则i++，j++，继续匹配下一个字符；
    - 如果S\[i\] = T\[j\]，则i++，j++，继续匹配下一个字符
    - 如果j != -1，且S\[i\] != T\[j\]，则i不变，j = next\[j\]。此举等价为失配时，接下来模式串T要相对于主串S向右移动j-next\[j\]位
    - 注：next数组指除当前字符外（注意不包括当前字符）的“公共 前后缀 最长长度”（前缀中和后缀中相同的最长的那一个子串的长度）
    - 前缀：不包含最后末尾字符，且首字母为第1个字符的子串；后缀：不包含第一个字符，且尾字符为最后一个字符的子串
    - next数组的生成可以参看对应链接
    
    

## 暴力匹配(BF)

In [ ]:
def BF(S, T):
    '''
    返回匹配上的S的起始位置
    '''
    i = 0
    j = 0
    maxS = len(S)
    maxT = len(T)
    while (i < maxS and j < maxT):
        if S[i] == T[j]:
            i += 1
            j += 1
        else:
            i = i-j+1 # 回到原来位置的下一个重新开始比较
            j = 0

    if j == maxT:
        return (i-j)
    else:
        return "匹配不上"
s = "abdcajsne"
t = "ajsne"
BF(s, t)

## KMP算法
kmp重点在于如何得到next数组
- https://blog.csdn.net/yyzsir/article/details/89462339

In [ ]:
def buildNext(S):
    Next = [None for _ in S]
    j = 0
    k = -1 # 表示“公共前后缀”的最长长度
    Next[j] = k # 初始化Next[0]=-1
    while j < len(S)-1:
        if k==-1 or S[j] == S[k]:   # 当k迭代到-1时都没有找到最短则应该让Next[j]=0，或者当S[k] == S[j]时，Next[j+1] = k+1
            j += 1
            k += 1
            Next[j] = k    # 记录下当前字符外的“公共前后缀最长长度”

            # ## 更好理解个更新方式是
            # Next[j+1] = k+1
            # j += 1
            # k += 1
        else:
            k = Next[k]    # 如果不相等，就找在前缀里面找
    return Next

def KMP(S, T):
    N = buildNext(S) # next列表
    i = 0
    j = 0
    maxS = len(S)
    maxT = len(T)
    while (i < maxS and j < maxT):
        if S[i] == T[j] or j == -1:
            i += 1  # S继续识别下一个字母是否匹配
            j += 1  
        else:
            j = N[j]    # j不用回到最开始0的位置，只用回到next[j]的位置，往后识别
    if j == maxT:
        return (i-j)
    else:
        return "匹配不上"
# s = "abaabca"
# t = "abca"
# KMP(s,t)

### 问题1
给定两个头节点的二叉树，返回其中一个是否为另一个的子结构(并不是查询是否为子树，而是值是否是另一个树的值得一部分)

- 将两个树进行先序序列化，查看两个序列化之后的列表是否具有包含关系采用KMP算法即可。
- 原因是：先序序列化的树结构是唯一的，只要有包含关系则一定能重构出相同的树结构。


In [ ]:
# 构造测试示例
class Node():
    def __init__(self,item):
        self.item=item
        self.left=None
        self.right=None
class Tree():
    def __init__(self):
        self._root = None 
    def addNode(self,item):
        node = Node(item)
        if self._root == None:
            self._root = node
            return

        cur = self._root
        queue = [cur]
        while queue:
            n = queue.pop(0)
            if n.left != None:
                queue.append(n.left)
            else:
                n.left = node
                break
            if n.right != None:
                queue.append(n.right)
            else:
                n.right=node
                break
tree1 = Tree()
tree1.addNode("可以")
tree1.addNode(1)
tree1.addNode(3)
tree1.addNode("12")
tree1.addNode("wei")
tree1.addNode("静静儿")

tree2 = Tree()
tree2.addNode(3)
tree2.addNode("静静儿")

In [ ]:
def savetree(root, a):
    if root == None :
        a.append(None)
    else:
        a.append(root.item)
        savetree(root.left, a)
        savetree(root.right, a)
    return "储存完毕"
a1 = []
a2 = []
savetree(tree1._root, a1)
savetree(tree2._root, a2)
print(a1)
print(a2)
KMP(a1, a2)

# bfprt算法
在无序数组中求第k小的数(没有第0小的数)。

- 方法一：
    - 利用荷兰国旗的算法思想，将数组分成3个部分，小于标定值，等于标定值，大于标定值的，我们可以得到在整个数组中，等于标定值的数在数组中排在第几名(因为左边全是小于标定值的数)。标定值是随机选的。
    - 如果我们的k不在等于标定值的范围内，则意味着第k大的数要么在左边区域，要么在右边区域，继续对不同区域做荷兰国旗问题就可以了。

- 方法二：bfprt算法
    - 后续仍然是荷兰国旗思想，但是选取标定值是通过挑选来得到的，而不是随机得到
    - 如何筛选这个标定值呢？
        - 将整个arr数组每五个位置分为一组，最一组的元素个数可以小于5
        - 在每个分组中由小到大排序(只用排组内)，取出每组排序后的中位数拿出来，构成长度大约为N/5的数组m。
        - 最后取出m的数组中的中位数(也就是第N/10小的数)，作为我的标定值来进行接下来的荷兰国旗算法。


In [ ]:
import random

def partition(arr, value):
    less = -1         # 小于value的最后一个位置     
    more = len(arr)   # 大于value的第一个位置
    L = 0             # 用于锁定等于value的值
    while L < more:   
        if arr[L] < value:
            less += 1
            arr[L], arr[less] = arr[less], arr[L]
            L += 1
        elif arr[L] > value:
            more -= 1
            arr[L], arr[more] = arr[more], arr[L]
        else:               
            L += 1
    return (less+1, more-1)

# 在arr中[L,R]区域中找到arr中的第k小的数(k=0,1,2,3,4...)
# 但是得保证[L,R]中一定有第k小的数
def getLessNumber(arr, k):
    if len(arr) == 1:
        return arr[0]
    value = arr[random.randint(0,len(arr))]
    (less, more) = partition(arr, value)    #荷兰国旗问题会改变原数组
    if less <= k and k <= more:
        return arr[k]
    elif k < less:
        return getLessNumber(arr[:less], k)
    elif k > more:
        return getLessNumber(arr[more+1:], k-more-1)

# 测试示例
arr = [1,333,12,23,41,2,3,4,4,5,12,234,3,3,24,567,5,2,1]
arrcopy = arr.copy()
getLessNumber(arrcopy, 4)#返回第5小的数

In [ ]:
def bfprt(arr, L, R, k):
    if L == R:
        return arr[L]
    value = getValue(arr[L:(R+1)])          #在指定范围内找到最优的标定值
    (less, more) = partition(arr, value)    #荷兰国旗问题会改变原数组
    if less <= k and k <= more:
        return arr[k]
    elif k < less:
        return bfprt(arr, L, less-1, k)
    elif k > more:
        return bfprt(arr, more+1, R, k)

def partition(arr, value):
    less = -1         # 小于value的最后一个位置     
    more = len(arr)   # 大于value的第一个位置
    L = 0             # 用于锁定等于value的值
    while L < more:   
        if arr[L] < value:
            less += 1
            arr[L], arr[less] = arr[less], arr[L]
            L += 1
        elif arr[L] > value:
            more -= 1
            arr[L], arr[more] = arr[more], arr[L]
        else:      
            L += 1
    return (less+1, more-1)


# 用来得到荷兰国旗算法需要的最佳值
def getValue(arr):
    m =[]
    mNumber = len(arr)//5 if len(arr)%5 == 0 else len(arr)//5+1
    for i in range(mNumber):
        L = i*5
        R = min((i+1)*5, len(arr))
        m.append(getMiddle(arr[L:R]))
    return bfprt(m, 0, len(m)-1, len(m)//2)

# 计算中位数
def getMiddle(arr):
    arrcopy = arr.copy()
    arrcopy.sort()
    return arrcopy[len(arr)//2]

# 测试示例
arr = [1,333,12,23,41,2,3,4,4,5,12,234,3,3,24,567,5,2,1]
arrcopy = arr.copy()
bfprt(arrcopy, 0, len(arrcopy)-1, 4)#返回第5小的数


# 并查集
我们可以用并查集来判断一个森林中有几棵树、某个节点是否属于某棵树等。并查集主要有两个功能，第一是查询x属于哪个集合，第二个是合并x和y使得他们俩在一个集合里。

分析:
- 每个“集合”(连通图)使用一个**代表元**来代表这个集合，就好比树的根节点root代表整颗树。主元不一样意味着x，y属于不同的“集合”。
- 使用一个数组pre表示上级列表，例如pre[i] = j，表示i的上级是j。如果pre[i] = i，意味着i是作为这个集合的**代表元**。根据初始化pre的不同，确定找到主元的条件也是不同的，例如：初始化pre的时候，每个位置定义是pre[k] = 0 ，因为开始的时候每个各元素之间都没有关系因此，每个元素都是自己的**代表元**，此时如果有pre[h] = 0 ，才说明主元为h。
- find（x）：找到x的主元，那就只能一级一级的往上找，先查看x的上级pre[x]（假设为j），再查看pre[j]（j的上级为i），再接着查看pre[i]（结果发现i
的上级是i，说明找到了主元了），因此x的主元就是i。
- join（x, y）：当i，j产生关系了，此时要考虑i和j是不是在同一个集合里（也就是i和j的代表元是否相同）。x的代表元fx，y的代表元fy都是可以通过find函数找到的
    - 若fx = fy，那么不用做任何操作。（因为x，y已经是一个团体里的了）
    - 若fx != fy，则可以定义fx是fy的上级（或者fy是fx的上级），从而更新pre[fy] = fx（或pre[fx] = fy），要注意我们更新的是x，y的代表元关系而不是x，y的关系


应用题型：
- 判断图的连接是否为环，求连通分支个数（每个子图中相互可以直接或间接到达），集合个数

In [ ]:
pre = [i for i in range(10)] # 定义了一个10人的并查集


## 查找函数
def find(x):
    while pre[x] != x: # 若pre[x]不等于x说明x不是主元，向上查找
        x = pre[x]
    return x


## 并函数
def join(x, y):
    fx = find(x)
    fy = find(y)
    if fx != fy:
        pre[fx] = fy # 定义fx的上级为fy

join(4, 2)
join(3, 2)
join(3, 5)


## 优化find()
我们在使用join(x,y)进行合并是随机指定上级（上面的例子是指定后者为上级），但是这种方式会使得最终得到的结构很不稳定，例如
```python
join(4, 2)
join(3, 2)
join(3, 5)
```
执行之后，我们查找3的代表元需要向上找2次，而使用
```python
join(4, 2)
join(3, 2)
join(5, 3)
```
我们查找3的代表元只需要向上找1次。因此如果能对查找路径进行优化该多好，于时我们可以使用递归函数，在查找路径的同时，更新当前位置的上级，直接更新为代表元。


In [ ]:
### 压缩路径的方法
def find(x):
    if pre[x] == x:
        return x
    else:
        pre[x] = find(pre[x])
        return pre[x]
